# Does resampling experiment help with predicting CbG sentences?

In [1]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import plydata as ply
from sqlalchemy import create_engine

from snorkel.labeling.analysis import LFAnalysis
from snorkeling_helper.generative_model_helper import (
    sample_lfs,
    run_generative_label_function_sampler,
)

warnings.filterwarnings("ignore")

In [2]:
username = "danich1"
password = "snorkel"
dbname = "pubmed_central_db"
database_str = (
    f"postgresql+psycopg2://{username}:{password}@/{dbname}?host=/var/run/postgresql"
)
conn = create_engine(database_str)

## Load the data

In [3]:
label_candidates_dir = Path("../label_candidates/output")
notebook_output_dir = Path("output/CbG")

In [4]:
L_abstracts = pd.read_csv(
    str(label_candidates_dir / Path("cg_abstract_train_candidates_resampling.tsv")),
    sep="\t",
)

print(L_abstracts.shape)
L_abstracts.head().T

(1292772, 109)


,0,1,2,3,4
LF_HETNET_DRUGBANK,-1,-1,-1,-1,-1
LF_HETNET_DRUGCENTRAL,-1,-1,-1,-1,-1
LF_HETNET_ChEMBL,-1,-1,-1,-1,-1
LF_HETNET_BINDINGDB,-1,-1,-1,-1,-1
LF_HETNET_PDSP_KI,-1,-1,-1,-1,-1
...,...,...,...,...,...
LF_GG_BICLUSTER_INCREASES_EXPRESSION,0,-1,-1,-1,-1
LF_GG_BICLUSTER_SIGNALING,1,-1,-1,-1,-1
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_CELL_PRODUCTION,0,-1,-1,-1,-1


In [5]:
L_full_text = pd.read_csv(
    str(label_candidates_dir / Path("cg_full_text_train_candidates_resampling.tsv")),
    sep="\t",
)

print(L_full_text.shape)
L_full_text.head().T

(2121209, 109)


,0,1,2,3,4
LF_HETNET_DRUGBANK,-1,-1,-1,-1,-1
LF_HETNET_DRUGCENTRAL,-1,-1,-1,-1,-1
LF_HETNET_ChEMBL,-1,-1,-1,-1,-1
LF_HETNET_BINDINGDB,-1,-1,-1,-1,-1
LF_HETNET_PDSP_KI,-1,-1,-1,-1,-1
...,...,...,...,...,...
LF_GG_BICLUSTER_INCREASES_EXPRESSION,-1,-1,0,-1,-1
LF_GG_BICLUSTER_SIGNALING,-1,-1,1,-1,-1
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_CELL_PRODUCTION,-1,-1,0,-1,-1


In [6]:
L_dev = pd.read_csv(
    str(label_candidates_dir / Path("cg_dev_test_candidates_resampling.tsv")), sep="\t"
) >> ply.query("split==7")
print(L_dev.shape)
L_dev.head().T

(500, 111)


,0,4,9,10,11
LF_HETNET_DRUGBANK,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_DRUGCENTRAL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_ChEMBL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_BINDINGDB,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_PDSP_KI,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1.0,-1.0,0.0,-1.0,-1.0
LF_GG_BICLUSTER_CELL_PRODUCTION,-1.0,-1.0,0.0,-1.0,-1.0
split,7.0,7.0,7.0,7.0,7.0
document_id,209829.0,178683.0,132454.0,19485.0,358873.0


In [7]:
L_test = pd.read_csv(
    str(label_candidates_dir / Path("cg_dev_test_candidates_resampling.tsv")), sep="\t"
) >> ply.query("split==8")
print(L_test.shape)
L_test.head().T

(500, 111)


,1,2,3,5,6
LF_HETNET_DRUGBANK,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_DRUGCENTRAL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_ChEMBL,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_BINDINGDB,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_PDSP_KI,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1.0,-1.0,-1.0,-1.0,-1.0
LF_GG_BICLUSTER_CELL_PRODUCTION,-1.0,-1.0,-1.0,-1.0,-1.0
split,8.0,8.0,8.0,8.0,8.0
document_id,291947.0,321677.0,339647.0,213797.0,30430.0


## Restort Based on the Candidate Abstracts

In [8]:
# Grab the document ids for resampling
sql = """
select cg_candidates.sentence_id, document_id, cg_candidates.candidate_id from sentence
inner join (
  select candidate.candidate_id, compound_gene.sentence_id from compound_gene
  inner join candidate on candidate.candidate_id=compound_gene.candidate_id
  ) as cg_candidates
on sentence.sentence_id = cg_candidates.sentence_id
"""
candidate_doc_df = pd.read_sql(sql, database_str)
candidate_doc_df.head()

,sentence_id,document_id,candidate_id
0,293853012,23111736,17520
1,361497023,24837142,35137
2,395950432,24746171,35735
3,481487831,30853913,42182
4,579412852,21454644,58965


In [9]:
dev_test_ids = (
    L_dev >> ply.select("document_id") >> ply.distinct() >> ply.pull("document_id")
)

filtered_candidate_id = (
    candidate_doc_df
    >> ply.query(f"document_id in {list(dev_test_ids)}")
    >> ply.pull("candidate_id")
)

In [10]:
sorted_train_df = pd.read_csv(
    str(notebook_output_dir / Path("cbg_dataset_mapper.tsv")), sep="\t"
)
sorted_train_df.head()

,document_id,dataset
0,23111736,train
1,24837142,train
2,24746171,train
3,30853913,tune
4,21454644,train


In [11]:
trained_documents = (
    sorted_train_df
    >> ply.inner_join(candidate_doc_df, on="document_id")
    >> ply.query("dataset=='train'")
    >> ply.pull("candidate_id")
)

In [12]:
filtered_L_abstracts = L_abstracts >> ply.query(
    f"candidate_id in {list(trained_documents)}"
)
print(filtered_L_abstracts.shape)
filtered_L_abstracts.head()

(904644, 109)


,LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HETNET_ChEMBL,LF_HETNET_BINDINGDB,LF_HETNET_PDSP_KI,LF_HETNET_US_PATENT,LF_HETNET_PUBCHEM,LF_HETNET_CG_ABSENT,LF_CG_CHECK_GENE_TAG,LF_DG_IS_BIOMARKER,...,LF_GG_NO_VERB,LF_GG_BICLUSTER_BINDING,LF_GG_BICLUSTER_ENHANCES,LF_GG_BICLUSTER_ACTIVATES,LF_GG_BICLUSTER_AFFECTS_EXPRESSION,LF_GG_BICLUSTER_INCREASES_EXPRESSION,LF_GG_BICLUSTER_SIGNALING,LF_GG_BICLUSTER_IDENTICAL_PROTEIN,LF_GG_BICLUSTER_CELL_PRODUCTION,candidate_id
1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,191
7,-1,-1,-1,-1,-1,-1,-1,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,449
8,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,450
10,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,897
11,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,898


In [13]:
filtered_L_full_text = L_full_text >> ply.query(
    f"candidate_id in {list(trained_documents)}"
)
print(filtered_L_full_text.shape)
filtered_L_full_text.head()

(1483352, 109)


,LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HETNET_ChEMBL,LF_HETNET_BINDINGDB,LF_HETNET_PDSP_KI,LF_HETNET_US_PATENT,LF_HETNET_PUBCHEM,LF_HETNET_CG_ABSENT,LF_CG_CHECK_GENE_TAG,LF_DG_IS_BIOMARKER,...,LF_GG_NO_VERB,LF_GG_BICLUSTER_BINDING,LF_GG_BICLUSTER_ENHANCES,LF_GG_BICLUSTER_ACTIVATES,LF_GG_BICLUSTER_AFFECTS_EXPRESSION,LF_GG_BICLUSTER_INCREASES_EXPRESSION,LF_GG_BICLUSTER_SIGNALING,LF_GG_BICLUSTER_IDENTICAL_PROTEIN,LF_GG_BICLUSTER_CELL_PRODUCTION,candidate_id
0,-1,-1,-1,-1,-1,-1,-1,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,170
1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,171
8,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,360
9,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,361
10,-1,-1,-1,-1,-1,-1,-1,0,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,362


## Construct the Grid Search

In [14]:
# Global Grid
epochs_grid = [250]
l2_param_grid = np.linspace(0.01, 5, num=5)
lr_grid = [1e-3]
grid = list(
    zip(epochs_grid * len(l2_param_grid), l2_param_grid, lr_grid * len(l2_param_grid))
)

# Abstracts

In [15]:
analysis_module = LFAnalysis(
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
)

abstract_lf_summary = analysis_module.lf_summary()
abstract_lf_summary.index = (
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
).columns.tolist()

abstract_lf_summary

,Polarity,Coverage,Overlaps,Conflicts
LF_HETNET_DRUGBANK,[1],0.080205,0.080205,0.080205
LF_HETNET_DRUGCENTRAL,[1],0.026783,0.026783,0.026783
LF_HETNET_ChEMBL,[1],0.030188,0.030188,0.030188
LF_HETNET_BINDINGDB,[1],0.004332,0.004332,0.004332
LF_HETNET_PDSP_KI,[1],0.003062,0.003062,0.003062
...,...,...,...,...
LF_GG_BICLUSTER_AFFECTS_EXPRESSION,[1],0.022110,0.022110,0.022110
LF_GG_BICLUSTER_INCREASES_EXPRESSION,[0],0.039291,0.039291,0.039291
LF_GG_BICLUSTER_SIGNALING,[1],0.045772,0.045772,0.045772
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,[0],0.027517,0.027517,0.027517


# Set up fields for resampling

In [16]:
lf_columns_base = list(L_abstracts.columns[0:9])
data_columns = []

# Abstracts

## Baseline

In [17]:
cbg_start = 0
cbg_end = 9
number_of_samples = 1

cbg_lf_range = range(cbg_start, cbg_end)
size_of_samples = [len(cbg_lf_range)]

In [18]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(cbg_lf_range),
            len(list(cbg_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [19]:
data_columns += run_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("cbg_training_marginals_baseline.tsv")
    ),
    curated_label="curated_cbg",
    entity_label="CbG_baseline",
    data_source="abstract",
)

100%|██████████| 1/1 [00:07<00:00,  7.66s/it]


## DaG

In [20]:
dag_start = 9
dag_end = 38

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, dag_end - dag_start]
number_of_samples = 50
dag_lf_range = range(dag_start, dag_end)

In [21]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(dag_lf_range),
            len(list(dag_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [22]:
data_columns += run_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="DaG",
    data_source="abstract",
)

100%|██████████| 50/50 [08:04<00:00,  9.70s/it]


## CtD

In [23]:
ctd_start = 38
ctd_end = 60

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, ctd_end - ctd_start]
number_of_samples = 50
ctd_lf_range = range(ctd_start, ctd_end)

In [24]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(ctd_lf_range),
            len(list(ctd_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [25]:
data_columns += run_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="CtD",
    data_source="abstract",
)

100%|██████████| 50/50 [06:48<00:00,  8.16s/it]


## CbG

In [26]:
cbg_start = 60
cbg_end = 80

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, cbg_end - cbg_start]
number_of_samples = 50
cbg_lf_range = range(cbg_start, cbg_end)

In [27]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(cbg_lf_range),
            len(list(cbg_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [28]:
data_columns += run_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("cbg_predicts_cbg_training_marginals.tsv")
    ),
    curated_label="curated_cbg",
    entity_label="CbG",
    data_source="abstract",
)

100%|██████████| 50/50 [09:35<00:00, 11.51s/it]


## GiG

In [29]:
gig_start = 80
gig_end = 108

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, gig_end - gig_start]
number_of_samples = 50
gig_lf_range = range(gig_start, gig_end)

In [30]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(gig_lf_range),
            len(list(gig_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [31]:
data_columns += run_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="GiG",
    data_source="abstract",
)

100%|██████████| 50/50 [08:00<00:00,  9.62s/it]


# Full Text

## DaG

In [32]:
dag_start = 9
dag_end = 38

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, dag_end - dag_start]
number_of_samples = 50
dag_lf_range = range(dag_start, dag_end)

In [33]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(dag_lf_range),
            len(list(dag_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [34]:
data_columns += run_generative_label_function_sampler(
    filtered_L_full_text,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="DaG",
    data_source="full_text",
)

100%|██████████| 50/50 [12:18<00:00, 14.76s/it]


## CtD

In [35]:
ctd_start = 38
ctd_end = 60

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, ctd_end - ctd_start]
number_of_samples = 50
ctd_lf_range = range(ctd_start, ctd_end)

In [36]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(ctd_lf_range),
            len(list(ctd_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [37]:
data_columns += run_generative_label_function_sampler(
    filtered_L_full_text,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="CtD",
    data_source="full_text",
)

100%|██████████| 50/50 [10:24<00:00, 12.50s/it]


## CbG

In [38]:
cbg_start = 60
cbg_end = 80

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, cbg_end - cbg_start]
number_of_samples = 50
cbg_lf_range = range(cbg_start, cbg_end)

In [39]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(cbg_lf_range),
            len(list(cbg_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [40]:
data_columns += run_generative_label_function_sampler(
    filtered_L_full_text,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="CbG",
    data_source="full_text",
)

100%|██████████| 50/50 [09:39<00:00, 11.59s/it]


## GiG

In [41]:
gig_start = 80
gig_end = 108

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, gig_end - gig_start]
number_of_samples = 50
gig_lf_range = range(gig_start, gig_end)

In [42]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(gig_lf_range),
            len(list(gig_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [43]:
data_columns += run_generative_label_function_sampler(
    filtered_L_full_text,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_cbg",
    entity_label="GiG",
    data_source="full_text",
)

100%|██████████| 50/50 [12:22<00:00, 14.84s/it]


# Write Performance to File

In [44]:
performance_df = pd.DataFrame.from_records(data_columns)
performance_df

,lf_num,auroc,aupr,bce_loss,sampled_lf_name,label_source,data_source,model,epochs,l2_param,lr_param
0,9,0.566458,0.162808,0.368750,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",CbG_baseline,abstract,tune,250,0.01,0.001
1,9,0.721370,0.269354,0.356180,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",CbG_baseline,abstract,test,250,0.01,0.001
2,1,0.570836,0.163111,0.367363,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",DaG,abstract,tune,250,0.01,0.001
3,1,0.721370,0.269354,0.356155,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",DaG,abstract,test,250,0.01,0.001
4,1,0.569523,0.162784,0.367401,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",DaG,abstract,tune,250,0.01,0.001
...,...,...,...,...,...,...,...,...,...,...,...
3997,28,0.815462,0.391434,2.593703,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",GiG,full_text,test,250,5.00,0.001
3998,28,0.632246,0.124106,4.485962,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",GiG,full_text,tune,250,5.00,0.001
3999,28,0.815462,0.391434,2.593703,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",GiG,full_text,test,250,5.00,0.001
4000,28,0.632246,0.124106,4.485962,"LF_HETNET_DRUGBANK,LF_HETNET_DRUGCENTRAL,LF_HE...",GiG,full_text,tune,250,5.00,0.001


In [45]:
(
    performance_df
    >> ply.call(
        "to_csv",
        str(Path("output/performance") / Path("CbG_performance.tsv")),
        index=False,
        sep="\t",
    )
)